In [46]:
import cv2
import numpy as np
import math

A1 = cv2.imread('watch.pgm', cv2.IMREAD_GRAYSCALE)
A1 = A1.astype(np.uint8)

if A1 is None:
    print('Image not found')

M1 = A1.shape[0]
N1 = A1.shape[1]

# Canvas twice the size of the original image
M2 = 2 * M1
N2 = 2 * N1
A2 = np.ones((M2, N2)) * 255  # Default intensity is white (255)

# Rotation angle in degrees by slide 29
theta = 25
theta_rad = theta * (math.pi / 180)  # Convert to radians

h_shear = -math.tan(theta_rad / 2)
v_shear = math.sin(theta_rad)

T1 = np.array([[1,0,-M1/2],
    [0,1,-N1/2],
    [0,0,1]])

T2 = np.array([[1,0,M2/2],
              [0,1,N2/2],
              [0,0,1]])

# Shear matrix for horizontal shear
Sh = np.array([[1, h_shear, 0],
                [0, 1, 0],
                [0, 0, 1]])

# Shear matrix for vertical shear
Sv = np.array([[1, 0, 0],
               [v_shear, 1, 0],
               [0, 0, 1]])


# Combine the shears
A = T2 @ Sh @ Sv @ Sh @ T1

# Calculate the inverse of A
A_inv = np.linalg.inv(A)

# Perform the transformation and interpolation
for i in range(M2):
    for j in range(N2):
        p = np.array([i, j, 1])

        q = np.matmul(A_inv, p)

        # Coordinates in A1
        x = q[0]
        y = q[1]

        ''' one-dimensional interpolation '''
        if 0 <= x < M1 and 0 <= y < N1:
            if abs(h_shear) > abs(v_shear):  # Horizontal shear, interpolate horizontally in the row
                k = int(np.floor(x))
                u = x - k  # Horizontal interpolation factor

                if 0 <= k < M1 - 1:
                    A2[i, j] = (1 - u) * A1[k, int(np.floor(y))] + u * A1[k + 1, int(np.floor(y))]

            else:  # Vertical shear, interpolate vertically in the column
                l = int(np.floor(y))
                v = y - l  # Vertical interpolation factor

                if 0 <= l < N1 - 1:
                    A2[i, j] = (1 - v) * A1[int(np.floor(x)), l] + v * A1[int(np.floor(x)), l + 1]

cv2.imwrite("Images/A1.png", A1)
cv2.imwrite("Images/A2.png", A2)


True

In [47]:
# Read and view an image
A1b = cv2.imread("watch.pgm", cv2.IMREAD_GRAYSCALE)

# Get height and width of source image
M1b = A1b.shape[0]
N1b = A1b.shape[1]

# Set height and width of target image
M2b = 2 * M1b
N2b = 2 * N1b
A2b = np.zeros((M2b,N2b))

# Set default intensity to white
A2b[:,:] = 255

# Translate image centre of A1b to origin
T1 = np.array(
[[1,0,-M1b/2],
 [0,1,-N1b/2],
 [0,0,1]])

# Rotate about origin by 25 degrees
theta = 25/180*math.pi
c = math.cos(theta)
s = math.sin(theta)
R = np.array(
[[c,-s, 0],
 [s, c, 0],
 [0, 0, 1]])

# Translate origin to image centre of A2b
T2 = np.array(
[[1,0,M2b/2],
 [0,1,N2b/2],
 [0,0,1]])

# Transformation that rotates A1b by theta about its centre
# and maps to the centre of A2b
Ab = np.matmul(T2,np.matmul(R,T1))

# Invert A
Ab = np.linalg.inv(Ab)

# Transformation with inverse mapping and bilinear interpolation
for i in range(0,M2b):
  for j in range(0,N2b):
    # coordinates of the (i,j)-th pixel in A2b
    x = i + 0.5
    y = j + 0.5
    # convert to homegeneous coordinates
    p = np.array([x,y,1])
    # transform with matrix A
    q = np.matmul(Ab,p)
    # coordinates in A1b
    x = q[0]
    y = q[1]
    # bilinear interpolation
    k = round(x) - 1
    l = round(y) - 1
    u = x - k - 0.5
    v = y - l - 0.5
    if ((k >= 0) and (k < M1b-1) and (l >= 0) and (l < N1b-1)):
      A2b[i,j] = round( (1-v) * ( (1-u)*A1b[k,l] + u*A1b[k+1,l] ) + v * ( (1-u)*A1b[k,l+1] + u*A1b[k+1,l+1] ) )

cv2.imwrite("Images/A1b.png",A1b)


True

In [48]:
difference = np.abs(A2b - A2)
mean = np.mean(difference)
print("Average difference:", mean)
# show the image differences
cv2.imwrite("Images/difference.png",difference)

Average difference: 1.3235108326242881


True